In [1]:
import os
os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
# New optuna study
import optuna
from Recommenders.SLIM.SLIMElasticNetRecommender_MultithreadWorking import MultiThreadSLIM_SLIMElasticNetRecommender

def objective_function(trial):

    params = {
        'topK' : trial.suggest_int('topK', 80, 800),
        'l1_ratio': 0.4033590645217344,
        'alpha': 9.521340863590419e-05
    }


    MAP = 0
    recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)
    recommender.fit(**params)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    print("Current MAP = {:.4f} with parameters {}".format(MAP, params))
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/SLIM/Slimnet_K_Best_2nd.csv", index = False)
    
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2024-11-30 23:29:03,769] A new study created in memory with name: no-name-0bafb40d-4c69-4ca9-bc11-246700a69687
Progress: 100%|██████████| 38121/38121 [09:39<00:00, 65.81items/s]


EvaluatorHoldout: Processed 35595 (100.0%) in 15.16 sec. Users per second: 2348


[I 2024-11-30 23:38:59,035] Trial 0 finished with value: 0.05997583930327103 and parameters: {'topK': 122}. Best is trial 0 with value: 0.05997583930327103.


Current MAP = 0.0600 with parameters {'topK': 122, 'l1_ratio': 0.4033590645217344, 'alpha': 9.521340863590419e-05}


Progress: 100%|██████████| 38121/38121 [10:09<00:00, 62.51items/s] 


...

Progress: 100%|██████████| 38121/38121 [15:52<00:00, 40.02items/s] 


EvaluatorHoldout: Processed 35595 (100.0%) in 16.16 sec. Users per second: 2202


[I 2024-12-01 10:39:58,335] Trial 58 finished with value: 0.060022675736959685 and parameters: {'topK': 321}. Best is trial 52 with value: 0.06006386664793552.


Current MAP = 0.0600 with parameters {'topK': 321, 'l1_ratio': 0.4033590645217344, 'alpha': 9.521340863590419e-05}


Progress:   0%|          | 0/38121 [03:05<?, ?items/s]


In [13]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.05739971393342457, 'l1_ratio': 0.9789616374877664, 'alpha': 9.950495272337068e-06}
